### Importing necessary stuff !

In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf

 ### Reading in data (and defining necessary constants ) !

In [ ]:
full_data = pd.read_csv('../input/train.csv')

In [ ]:
LABEL = 'label'

In [ ]:
NUM_PREDICTOR_COLS = 784
PREDICTOR_COLS = ['pixel' + str(i) for i in range(NUM_PREDICTOR_COLS)]

In [ ]:
REL_COLUMNS = PREDICTOR_COLS.copy()
REL_COLUMNS.append('label')

### Checking for null values.

In [ ]:
full_data.isnull().values.any()

### Split into training and validation sets

In [ ]:
LEN_TRAIN_SET = int(0.8 * len(full_data))
train_data = full_data[0:LEN_TRAIN_SET]
validation_data = full_data[LEN_TRAIN_SET:len(full_data)]

In [ ]:
assert(len(train_data) + len(validation_data) == len(full_data))

In [ ]:
len(full_data)

In [ ]:
len(train_data)

In [ ]:
len(validation_data) 

### Fit model on training data


In [ ]:
def make_train_input_fn(df, num_epochs):
    return tf.estimator.inputs.pandas_input_fn(
        x=df,
        y=df[LABEL],
        shuffle=True,
        num_epochs=num_epochs)

In [ ]:
def make_eval_input_fn(df):
    return tf.estimator.inputs.pandas_input_fn(
        x=df,
        y=df[LABEL],
        shuffle=False)

In [ ]:
def make_prediction_input_fn(df):
    return tf.estimator.inputs.pandas_input_fn(
        x=df,
        y=None,
        shuffle=False)    

In [ ]:
def make_features():
    input_cols = [tf.feature_column.numeric_column(col) for col in PREDICTOR_COLS]
    return input_cols

In [ ]:
X_train = train_data[PREDICTOR_COLS]
Y_train = np.ravel(train_data[['label']])

In [ ]:
OUTDIR = 'classification_outputs'
import shutil
shutil.rmtree(OUTDIR, ignore_errors=True)

In [ ]:
N_CLASSES = 10

In [ ]:
HIDDEN_UNITS = [256, 32]

In [ ]:
model = tf.estimator.DNNClassifier(feature_columns=make_features(),
                                   hidden_units=HIDDEN_UNITS,
                                   n_classes=N_CLASSES,
                                   dropout=0.2,
                                   optimizer=tf.train.AdamOptimizer(1e-3),
                                   model_dir=OUTDIR)


In [ ]:
model.train(input_fn=make_train_input_fn(train_data[REL_COLUMNS], num_epochs=100))    

In [ ]:
metrics = model.evaluate(input_fn = make_eval_input_fn(train_data[REL_COLUMNS]))
print(metrics['accuracy'])

In [ ]:
metrics = model.evaluate(input_fn = make_eval_input_fn(validation_data[REL_COLUMNS]))
print(metrics['accuracy'])

### Making predictions on test data

In [ ]:
test_data = pd.read_csv('../input/test.csv')

In [ ]:
test_predictions = model.predict(input_fn=make_prediction_input_fn(test_data[PREDICTOR_COLS]))

In [ ]:
test_data['Label'] = [int(i['classes'][0]) for i in test_predictions]

In [ ]:
test_data['ImageId'] = np.arange(1, len(test_data)+1)

In [ ]:
test_data[['ImageId', 'Label']].to_csv('submission_dnn_tensorflow.csv', index=False)